<a href="https://colab.research.google.com/github/bbberylll/ESAA/blob/main/%EC%88%98%EC%83%81%EC%9E%91_%EB%A6%AC%EB%B7%B0_M09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New York City Taxi Trip Duration - 수상작 리뷰 (by Gabor Fodor)

대회 주제:주어진 택시 승차 및 하차 정보를 기반으로 trip_duration을 예측하는 회귀 문제

https://www.kaggle.com/competitions/nyc-taxi-trip-duration

데이터 특성:
1. 약 145만 건의 택시 승차 기록
2. 주요 변수: pickup_datetime, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, passenger_count, store_and_fwd_flag
3. 타겟 변수인 trip_duration은 초 단위로 측정

## 적용 기법

모델: XGBoost Regressor

전처리:
1. 로그 변환: trip_duration에 로그 변환 적용
2. 이상치 제거: 비정상적으로 짧거나 긴 여행 시간 제거

특성 엔지니어링:
1. 거리 계산: Haversine 거리, 맨해튼 거리, 베어링 등
2. 시간 관련 변수 생성: 요일, 시간대, 월 등
3. 클러스터링: KMeans를 사용하여 픽업 및 드롭오프 위치를 클러스터링하여 지역 정보 생성

## 문제 접근 방식

### 데이터 전처리:
1. 로그 변환을 통해 타겟 변수의 분포를 정규화
2. 이상치 제거를 통해 모델의 학습 안정성 향상

### 특성 엔지니어링:
1. 거리 및 방향 정보를 통해 공간적 특성 반영
2. 시간 관련 변수를 통해 시간대별 패턴 캡처
3. 클러스터링을 통해 지역적 특성 반영

### 모델링:
1. XGBoost를 사용하여 회귀 모델 학습
2. 교차 검증을 통해 모델의 일반화 성능 평가

## 코드 흐름 요약
(1) 데이터 로딩 및 결측치 처리

```
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train['dataset_type'] = 'train'
test['dataset_type'] = 'test'
data = pd.concat([train, test], axis=0)
```

(2) 로그 변환 및 이상치 제거
```
data['trip_duration_log'] = np.log1p(data['trip_duration'])
data = data[(data['trip_duration'] > 60) & (data['trip_duration'] < 3600)]
```

(3) 거리 및 방향 계산
```
def haversine_distance(lat1, lon1, lat2, lon2):
    # 거리 계산 함수 구현
    return distance

data['haversine_distance'] = haversine_distance(
    data['pickup_latitude'], data['pickup_longitude'],
    data['dropoff_latitude'], data['dropoff_longitude']
)
```

(4) 시간 관련 변수 생성
```
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['pickup_hour'] = data['pickup_datetime'].dt.hour
data['pickup_day_of_week'] = data['pickup_datetime'].dt.dayofweek
```

(5) 클러스터링을 통한 지역 정보 생성
```
from sklearn.cluster import KMeans

coords = np.vstack((data[['pickup_latitude', 'pickup_longitude']].values,
                    data[['dropoff_latitude', 'dropoff_longitude']].values))
kmeans = KMeans(n_clusters=100, random_state=42).fit(coords)
data['pickup_cluster'] = kmeans.predict(data[['pickup_latitude', 'pickup_longitude']])
data['dropoff_cluster'] = kmeans.predict(data[['dropoff_latitude', 'dropoff_longitude']])
```

(6) 모델 학습
```
from xgboost import XGBRegressor

features = ['haversine_distance', 'pickup_hour', 'pickup_day_of_week',
            'pickup_cluster', 'dropoff_cluster', 'passenger_count']
X_train = data[data['dataset_type'] == 'train'][features]
y_train = data[data['dataset_type'] == 'train']['trip_duration_log']

model = XGBRegressor()
model.fit(X_train, y_train)
```

## 차별점 (유의미한 포인트)

1. 공간적 특성 반영: 거리 및 방향 계산을 통해 픽업 및 드롭오프 위치 간의 공간적 특성 반영
2. 시간대별 패턴 캡처: 시간 관련 변수를 통해 교통 패턴의 시간적 변동성 반영
3. 지역 정보 활용: 클러스터링을 통해 지역적 특성을 모델에 반영하여 예측 성능 향상